In [4]:
%reload_ext autoreload
%autoreload 2
import json
import sys
sys.path.append("../..")
from pathlib import Path
from data_processing.online_coordinate_data.create_dataset import *
import matplotlib.pyplot as plt
from hwr_utils import utils

import numpy as np
from hwr_utils.stroke_plotting import draw_from_gt

import json
## GT text for online 
json.load(Path("../../data/prepare_online_data/online_augmentation.json").open())

## Stroke
xml_path = Path("/media/data/GitHub/simple_hwr/data/prepare_online_data/line-level-xml/lineStrokes/a01/a01-000/a01-000u-01.xml")
stroke_list, _ = read_stroke_xml(xml_path)

In [5]:
print(stroke_list)

[{'x': [1073, 1072, 1066, 1052, 1030, 1009, 994, 980, 971, 968, 966, 972, 978, 991, 1003, 1016, 1021, 1030, 1040, 1049, 1055, 1058, 1064, 1071, 1071, 1074, 1083, 1097, 1114, 1134, 1158, 1181, 1200, 1218, 1220, 1223, 1221, 1220, 1215, 1204], 'y': [-1058, -1085, -1117, -1152, -1196, -1242, -1286, -1317, -1336, -1344, -1339, -1340, -1320, -1298, -1266, -1231, -1184, -1143, -1108, -1077, -1049, -1021, -1006, -1006, -1006, -1013, -1042, -1082, -1124, -1172, -1225, -1280, -1327, -1365, -1373, -1376, -1372, -1369, -1353, -1330], 'time': [0.0, 0.02, 0.03, 0.05, 0.07, 0.08, 0.09, 0.11, 0.13, 0.14, 0.15, 0.17, 0.19, 0.2, 0.22, 0.23, 0.25, 0.26, 0.28, 0.29, 0.31, 0.32, 0.33, 0.35, 0.37, 0.38, 0.4, 0.41, 0.43, 0.44, 0.46, 0.47, 0.49, 0.5, 0.52, 0.53, 0.55, 0.56, 0.58, 0.59]}, {'x': [1176, 1175, 1171, 1169, 1158, 1139, 1118, 1092, 1065, 1041, 1024, 1015, 1010, 1014], 'y': [-1237, -1233, -1229, -1230, -1230, -1227, -1228, -1229, -1233, -1236, -1239, -1243, -1239, -1243], 'time': [0.65, 0.67, 0.69, 0

In [11]:
import torch
from loss_module import losses
from loss_module.losses import DTWLoss

def dtw_with_reverse(self, preds, targs, label_lengths, **kwargs):
    loss = 0
    for i in range(len(preds)):  # loop through BATCH
        a, b   = DTWLoss.dtw_single((preds[i], targs[i]), dtw_mapping_basis=self.dtw_mapping_basis)
        ar, br = DTWLoss.dtw_single((preds[i], targs[i]), dtw_mapping_basis=self.dtw_mapping_basis)

        # LEN X VOCAB
        if self.method=="normal":
            pred = preds[i][a, :][:, self.loss_indices]
            targ = targs[i][b, :][:, self.loss_indices]
        else:
            raise NotImplemented

        ## !!! DELETE THIS
        if self.barron:
            loss += (self.barron(pred - targ) * self.subcoef).sum()  # AVERAGE pointwise loss for 1 image
        elif True:
            loss += (abs(pred - targ) * self.subcoef).sum()  # AVERAGE pointwise loss for 1 image

        if self.cross_entropy_indices:
            pred = preds[i][a, :][:, self.cross_entropy_indices]
            targ = targs[i][b, :][:, self.cross_entropy_indices]

            if self.relativefy:
                targ = relativefy_torch(targ)
                targ[0][:, self.cross_entropy_indices] = 1 # first point is a start point
            loss += BCEWithLogitsLoss(pred, targ).sum() * .1  # AVERAGE pointwise loss for 1 image

    return loss  # , to_value(loss)



#diff  = torch.sum(torch.abs(pred.reshape(-1, 2) - targ.reshape(-1, 2)), axis=1)
#diff2 = torch.sum(torch.abs(pred.reshape(-1, 2) - torch.flip(targ.reshape(-1, 2), dims=(1,))), axis=1)
            

In [17]:
targs = np.array([[1,2,3,4,6,7,8,9],[1,2,3,4,6,7,8,9],[1,0,0,1,0,0,0,0]]).astype(np.float64).transpose()
preds = np.array([[1,2,3,9,8,7,6,5],[1,2,3,9,8,7,6,5],[1,0,0,1,0,0,0,0]]).astype(np.float64).transpose()

a, b   = DTWLoss.dtw_single((preds, targs), dtw_mapping_basis=[0,1])
ar, br = DTWLoss.dtw_single((preds, targs), dtw_mapping_basis=[0,1])


print(a,ar)

[0, 0, 1, 2, 2, 3, 4, 5, 6, 7] [0, 0, 1, 2, 2, 3, 4, 5, 6, 7]
